In [3]:
import src.util as utils
import pandas as pd

In [4]:
konfig = utils.load_params(str(utils.get_dir()) + utils.get_params())

In [24]:
def ambil_data():
    X_train = utils.pickle_load(str(utils.get_dir())+utils.cek_path_os(konfig["train_set_path"][0]))
    y_train = utils.pickle_load(str(utils.get_dir())+utils.cek_path_os(konfig["train_set_path"][1]))
    
    train_set = pd.concat([X_train, y_train], axis = 1)
    
    return train_set

In [6]:
def cek_outlier(dataset, pengkali):
    interkuartil = pengkali * (dataset["HARGA"].quantile(0.75) - dataset["HARGA"].quantile(0.25))
    batas_atas = dataset["HARGA"].quantile(0.75) + interkuartil
    data_outlier = dataset[dataset["HARGA"] > batas_atas]
    
    return data_outlier

In [7]:
def cek_data(dataset, kolom, rentang, pengkali):
    #fungsi ini awal2 akan memecah dataset dalam beberapa bagian sesuai rentang yang dimasukkan
    #kemudian akan dicek outliernya pada masing-masing blok tersebut
    data_outlier = pd.DataFrame()
    blok = int(dataset[kolom].max() / rentang)
    for i in range (1, blok):
        data_blok = dataset[dataset[kolom] < rentang * i]
        data_blok_cek_outlier = cek_outlier(data_blok, pengkali)
        data_outlier = pd.concat([data_outlier, data_blok_cek_outlier])
        #mengeluarkan data_blok dari dataset lama
        dataset = dataset[~dataset.index.isin(data_blok.index)]
    cek_outlier(dataset, pengkali)    
    return data_outlier

In [25]:
def hapus_outlier(dataset, outlier):
    try: 
        return dataset[~dataset.index.isin(outlier.index)]
    except:
        return dataset

In [27]:
train_set = ambil_data()

In [34]:
outlier_LB = cek_data(train_set, "LB", konfig["blok_LB"][0], konfig["blok_LB"][1])
outlier_LB

,LB,LT,KT,KM,GRS,HARGA
532,140,246,6,6,1,16000000000
470,400,1075,4,3,4,55000000000
283,300,200,5,7,4,65000000000


In [35]:
outlier_LT = cek_data(train_set, "LT", konfig["blok_LT"][0], konfig["blok_LT"][1])
outlier_LT

,LB,LT,KT,KM,GRS,HARGA
283,300,200,5,7,4,65000000000


In [36]:
train_set.shape

(707, 6)

In [37]:
train_set_clean = hapus_outlier(train_set, outlier_LB)
train_set_clean.shape

(704, 6)

In [38]:
train_set_clean = hapus_outlier(train_set_clean, outlier_LT)
train_set_clean.shape

(704, 6)

In [39]:
utils.pickle_dump(train_set_clean[konfig["prediktor"]], konfig["train_clean_set_path"][0])
utils.pickle_dump(train_set_clean[konfig["label"]], konfig["train_clean_set_path"][1])